In [1]:
from pathlib import Path

from langchain import LLMMathChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

from trulens_eval import Tru, FeedbackMode
from trulens_eval.tru_agent import TruAgent, OpenAiFunctionsMessagesExtractor

In [2]:
tru = Tru()

In [3]:
tru.start_dashboard(force=True)

Force stopping dashboard ...
Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.178.30:8501 .


In [4]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]

agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [5]:
app = TruAgent(
    app=agent,
    app_id="OpenAiFunctions",
    messages_extractor=OpenAiFunctionsMessagesExtractor(agent),
)

out, rec, session = app.call_with_session(inputs={"input": "what is the square root of 49"})



> Entering new AgentExecutor chain...

Invoking: `Calculator` with `sqrt(49)`




> Entering new LLMMathChain chain...
sqrt(49)```text
sqrt(49)
```
...numexpr.evaluate("sqrt(49)")...

Answer: 7.0
> Finished chain.
Answer: 7.0The square root of 49 is 7.

> Finished chain.


In [6]:
tru.db.get_messages(session.session_id)

,ts,source,label,content,metadata_
0,2023-08-09 19:11:40.337369,user,Input,what is the square root of 49,{'agent_type': 'openai-functions'}
1,2023-08-09 19:11:41.752353,assistant,Action[Calculator],sqrt(49),"{'agent_type': 'openai-functions', 'tool': {'n..."
2,2023-08-09 19:11:42.834902,system,Observation,Answer: 7.0,{'agent_type': 'openai-functions'}
3,2023-08-09 19:11:43.791753,assistant,AgentFinish,The square root of 49 is 7.,{'agent_type': 'openai-functions'}
